## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [37]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2024-11-22 11:22:39--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2024-11-22 11:22:40--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log.1’

auto_purchase.log.1 100%[===================>] 448.65K  --.-KB/s    in 0.03s   

2024-11-22 11:22:40 (12

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[500:600]:
    print(line)

INFO  | 2022-12-26 00:01:03,151  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-12-26, количество людей с автопродлением подписки: 0

INFO  | 2022-12-26 00:01:03,154  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-12-27 00:01:02,245  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-12-27 00:01:02,266  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-12-27, количество людей с автопродлением подписки: 0

INFO  | 2022-12-27 00:01:02,269  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-12-28 00:01:02,277  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-12-28 00:01:02,298  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-12-28, количество людей с автопродлением подписки: 1

INFO  | 2022-12-28 00:01:02,302  | f

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:
def count_success_and_failure(file_path):
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()
    success_count = 0
    error_count = 0
  for l in lines:
    if 'ERROR' in l:
      error_count += 1
    if 'Обновляем подписку пользователю' in l:
      success_count += 1
  return (success_count - error_count, error_count)

count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [ ]:
def auto_renewal_sub(log_file_path):
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()
  #отбираем дату и кол-во в список кортежей
  tabel = []
  import datetime as d
  for l in lines:
    for l in lines:
      if 'Cегодня ' in l:
        tabel.append((d.datetime.strptime(l[8:18], "%Y-%m-%d").date(),int(l[-3:-1])))

  #убираем дубли (оставляем с максимальным значением)
  tabel = list(set(sorted(tabel, key = lambda x: x[1] , reverse= True)))

  #упорядочиваем по дате
  tabel.sort(key=lambda x: x[0])

  #убираем данные по датам, оставляем только данные для анализа в хронологическом порядке
  for i in range(len(tabel)):
    tabel[i] = tabel[i][1]

  #расчет скользящего среднего (от первого показателя по текущий)
  summ = 0
  num = 0
  SMA = []
  for t in tabel:
    summ += t
    num += 1
    SMA.append(round(summ/num, 2))

  #расчет скользящей медианы (от первого показателя по текущий)
  MMA = []
  import statistics
  for i in range(len(tabel)):
    rez = statistics.median(tabel[:i+1])
    MMA.append(round(rez))
#записываем расчеты в файл
  with open ('auto_renewal_sub.txt','w') as output:
    print(f'Среднее: {SMA}', file=output)
    print(f'Медиана: {MMA}', file=output, end='')



auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2024-11-22 10:28:59--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.116.4
Connecting to gist.github.com (gist.github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2024-11-22 10:28:59--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2024-11-22 10:2

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [38]:
def sub_renewal_by_day(file_path):
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()
  #собираем в список все записи об успешном обновлении подписов
  cnt = []
  import datetime as d
  for i in range(len(lines)):
    if i != len(lines)-1:
      if 'Обновляем подписку пользователю' in lines[i] and 'ERROR' not in lines[i+1]:
        cnt.append(d.datetime.strptime(lines[i][8:18], '%Y-%m-%d').weekday())
    else:
      if 'Обновляем подписку пользователю' in lines[i]:
        cnt.append(d.datetime.strptime(lines[i][8:18], '%Y-%m-%d').weekday())
  #словарь для дней недели
  weekdays = {0:'Понедельник', 1:'Вторник', 2:'Среда', 3:'Четверг', 4:'Пятница', 5:'Суббота', 6:'Воскресенье'}
  #записываем в файл статистику
  with open('weekdays.txt', 'w') as output:
    print('Количество обновлений подписки по дням недели:', file=output, end = '')
    for i in range(7):
      print('', file=output)
      print(f'{weekdays[i]}: {cnt.count(i)}', file=output, end = '')

sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [39]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2024-11-22 11:23:05--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2024-11-22 11:23:06--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt.1’

cor_weekdays.txt.1  100%[===================>]     238  --.-KB/s    in 0s      

2024-11-22 11:23:06 (10.5 MB/s) - ‘co

In [40]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
